In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install scikit-surprise==1.1.0

In [0]:
import pandas as pd
from surprise import SVD
from surprise import NMF
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
import numpy as np
from sklearn import metrics
from scipy.spatial.distance import correlation, cosine
from sklearn.neighbors import NearestNeighbors
from surprise.model_selection import train_test_split
import statistics
import pickle

In [0]:
ratings = pd.read_csv("/content/drive/My Drive/ACMS/goodbooks-10k-master/ratings.csv")
books = pd.read_csv("/content/drive/My Drive/ACMS/goodbooks-10k-master/books.csv")

In [0]:
user_item_matrix = ratings.pivot(index='user_id', columns='book_id', values='rating').fillna(0)

In [0]:
user_item_matrix

book_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,9961,9962,9963,9964,9965,9966,9967,9968,9969,9970,9971,9972,9973,9974,9975,9976,9977,9978,9979,9980,9981,9982,9983,9984,9985,9986,9987,9988,9989,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,4.0,0.0,5.0,4.0,0.0,2.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,5.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,5.0,5.0,5.0,4.0,5.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,5.0,0.0,4.0,4.0,0.0,4.0,4.0,0.0,5.0,4.0,0.0,4.0,3.0,0.0,0.0,0.0,5.0,2.0,0.0,5.0,1.0,5.0,5.0,5.0,3.0,5.0,3.0,0.0,0.0,0.0,4.0,3.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53420,4.0,5.0,3.0,0.0,2.0,0.0,0.0,0.0,4.0,3.0,3.0,0.0,0.0,3.0,3.0,4.0,3.0,0.0,5.0,2.0,0.0,3.0,0.0,0.0,5.0,3.0,0.0,2.0,3.0,3.0,5.0,3.0,3.0,1.0,0.0,0.0,4.0,2.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53421,4.0,5.0,0.0,5.0,4.0,0.0,4.0,0.0,5.0,0.0,5.0,5.0,0.0,4.0,0.0,0.0,4.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,4.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53422,4.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,5.0,5.0,4.0,5.0,0.0,5.0,5.0,5.0,0.0,5.0,0.0,4.0,0.0,0.0,4.0,5.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
# Main Model
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute') 
model_knn.fit(user_item_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [0]:
# filename = 'knn_model.sav'
# pickle.dump(model_knn, open(filename, 'wb'))

In [0]:
# #This function finds k similar users given the user_id and ratings matrix M
# #Note that the similarities are same as obtained via using pairwise_distances
# def findksimilarusers(new_user, ratings, metric = correlation, k=2):
#     similarities=[]
#     indices=[]
#     model_knn = NearestNeighbors(metric = metric, algorithm = 'brute') 
#     model_knn.fit(ratings)

#     # 
#     distances, indices = model_knn.kneighbors(new_user, n_neighbors = k+1)
#     similarities = 1-distances.flatten()
#     print('{0} most similar users for new User:\n'.format(k))
#     for i in range(0, len(indices.flatten())):

#             print('{0}: User {1}, with similarity of {2}'.format(i, indices.flatten()[i]+1, similarities.flatten()[i]))
            
#     return similarities,indices

In [0]:
class MyPack:
# n = number of books to be recommended
  def predict__collab(new_list,n):

   #This function finds k similar users given the user_id and ratings matrix M
    #Note that the similarities are same as obtained via using pairwise_distances
    def findksimilarusers(new_user, ratings, metric = correlation, k=2):
      similarities=[]
      indices=[]

      # 
      distances, indices = model_knn.kneighbors(new_user, n_neighbors = k+1)
      similarities = 1-distances.flatten()
      print('{0} most similar users for new User:\n'.format(k))
      for i in range(0, len(indices.flatten())):

              print('{0}: User {1}, with similarity of {2}'.format(i, indices.flatten()[i]+1, similarities.flatten()[i]))
              
      return similarities,indices

    # Converting new user list to new user df
    new_user = pd.DataFrame(new_list).T

    # Finding books which user has already read
    book_not_read = new_user.iloc[0][new_user.iloc[0] == 0]
    book_not_read = book_not_read.index+1

    # Finding similar users to the new user
    similarities,indices = findksimilarusers(new_user,user_item_matrix, metric='correlation')
    index_list = indices[0]
    
    # Making new dataframe of similar users
    similar_user_df = pd.DataFrame()
    for i in index_list:
      similar_user_df = similar_user_df.append(user_item_matrix.iloc[i:i+1])

    # Finding sum of ratings of similar users for all the books
    arr = []
    for i in similar_user_df:
      arr.append(similar_user_df[i].sum())

    # Making dataframe of ratings
    rat = pd.DataFrame(arr).T

    # Making final dataframe
    final_rat = pd.DataFrame(rat.stack())
    final_rat = final_rat.reset_index(level=1)
    final_rat.rename(columns={final_rat.columns[0]:'book_id',final_rat.columns[1]:'rating'},inplace=True)
    final_rat.reset_index(drop=True,inplace=True)

    # Sorting the final ratings
    sorted_df = final_rat.sort_values(by='rating',axis=0,ascending=False)
    sorted_df.reset_index(drop=True,inplace=True)

    # As book_id are from 0 to 9999 therefore removing book_id = 1 instead of 2. 
    sorted_df = sorted_df[sorted_df.apply(lambda x: x['book_id']+1 in book_not_read, axis=1)]

    # Making a list of books to be recommended
    book_rec = []
    for i in range(n):
      book_rec.append(sorted_df['book_id'].iloc[i]+1)

    # Making final result dataframe
    result_df = pd.DataFrame()
    for i in book_rec:
      result_df = result_df.append(books.iloc[i-1:i])
    result_df.reset_index(drop=True,inplace = True)
    full_result = result_df
    result_df.drop(result_df.columns.difference(['book_id','goodreads_book_id','isbn','title','authors','average_rating','image_url']),1, inplace=True)
    print(result_df)

    # Storing isbn of recommended books in a list
    isbn_list = []
    for i in range(n):
      isbn_list.append(result_df['isbn'].iloc[i])

    return(isbn_list)